In [24]:
# Import Various Libraries, including Tweepy, a Python library for the Twitter API,
# and TextBlob, a library with a sentiment analysis method.
import os
import requests
import pandas as pd
import hvplot.pandas
from dotenv import load_dotenv
from textblob import TextBlob 
#%matplotlib inline
import tweepy
#import twitter_reader.py
load_dotenv()
##########Micheal imports
import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import json
import numpy as np
%matplotlib inline
from datetime import date
from datetime import timedelta
pd.options.display.float_format = '{:.2f}'.format

In [25]:
#Pass API Keys to Twitter API and build Tweepy API handler object.
consumer_key = os.getenv('TWITTER_API_KEY')
consumer_secret = os.getenv('TWITTER_SECRET_KEY')
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

TweepError: Expected token_type to equal "bearer", but got None instead

In [3]:
#Read in Tweets Data
df_tweets = pd.read_csv('../data/raw_data/cleaner_tweets.csv')
df_tweets.head()

FileNotFoundError: [Errno 2] File ../data/raw_data/cleaner_tweets.csv does not exist: '../data/raw_data/cleaner_tweets.csv'

In [4]:
#Slicing the DataFrame of tweets by keyword category.
#df_tweets.reset_index(inplace=True)
df_tweets['time']= pd.to_datetime(df_tweets['time'],format= '%b:%d:%Y',infer_datetime_format=True ).dt.date
df_tweets = df_tweets.set_index('time')
df_bitcoin_tweets = df_tweets.loc[df_tweets.category.str.contains('bitcoin')]
df_ethereum_tweets = df_tweets.loc[df_tweets.category.str.contains('ethereum')]
df_litecoin_tweets = df_tweets.loc[df_tweets.category.str.contains('litecoin')]
df_generic_tweets = df_tweets.loc[df_tweets.category.str.contains('generic')]
df_tweets.head()

NameError: name 'df_tweets' is not defined

In [5]:
#Calculating Rolling Sentiment Polarity average for each tweet category.
df_bitcoin_tweets_rolling = df_bitcoin_tweets['Polarity Rating'].rolling(window=50).mean()
df_ethereum_tweets_rolling = df_ethereum_tweets['Polarity Rating'].rolling(window=50).mean()
df_litecoin_tweets_rolling = df_litecoin_tweets['Polarity Rating'].rolling(window=50).mean()
df_generic_tweets_rolling = df_generic_tweets['Polarity Rating'].rolling(window=50).mean()
df_bitcoin_tweets_rolling

NameError: name 'df_bitcoin_tweets' is not defined

In [6]:
from bokeh.models.formatters import DatetimeTickFormatter

formatter = DatetimeTickFormatter(months='%b %Y')
bitcoin_roller_graph = df_bitcoin_tweets_rolling.hvplot(x='time',y=['Polarity Rating'],xticks=12,color='red',width=1000, height=500,xformatter=formatter)
ethereum_roller_graph = df_ethereum_tweets_rolling.hvplot(x='time',y=['Polarity Rating'],xticks=12,color='blue',width=1000, height=500,)
generic_roller_graph = df_generic_tweets_rolling.hvplot(x='time',y=['Polarity Rating'],xticks=12,color='black',width=1000, height=500,)
litecoin_roller_graph = df_litecoin_tweets_rolling.hvplot(x='time',y=['Polarity Rating'],xticks=12,color='yellow',legend='top_left',width=1000, height=500,)
tweets_graph = df_generic_tweets.hvplot(width=1000, height=500,x='time', y='Polarity Rating',rot=90,xticks=12,hover_cols = ['twitter_user','text'],
                                        kind='scatter',aggregator='mean',color='twitter_user',xformatter=formatter)
tweets_graph*generic_roller_graph

NameError: name 'df_bitcoin_tweets_rolling' is not defined

In [7]:
bitcoin_roller_graph * ethereum_roller_graph * generic_roller_graph * litecoin_roller_graph

NameError: name 'bitcoin_roller_graph' is not defined

In [8]:
#Using recent search to generate recent tweets from anyone that include our keywords.
#Define Search Term Library
dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin',],
                     'generic':['cryptocurrency','blockchain'],
                     'litecoin':['litecoin','ltc','#litecoin', '#ltc'],
                     'ethereum':['ethereum','eth','#ethereum','#eth']   }
df_tweets_live = pd.DataFrame(dict_tweet_structure)
query = query_creator(dict_search_terms['generic']+dict_search_terms['bitcoin']+dict_search_terms['litecoin']+dict_search_terms['ethereum'])
print(query)
live_results = tweet_search_free(query, 1000)
for tweets in live_results:
    df_tweets_live = tweet_dataframe_append(live_results, df_tweets_live)
    

NameError: name 'dict_tweet_structure' is not defined

In [9]:
polarity_test = df_tweets_live
polarity_test.head()

NameError: name 'df_tweets_live' is not defined

In [10]:
polarity_test['time'] = pd.to_datetime(polarity_test['time'],format= '%M:%S',infer_datetime_format=True ).dt.time

NameError: name 'polarity_test' is not defined

In [11]:
rolling_poll = polarity_test['Polarity Rating'].rolling(window=20).mean()
end = pd.to_timedelta([str(polarity_test['time'].iloc[0])])
start = pd.to_timedelta([str(polarity_test['time'].iloc[-1])])
recent_tweets_duration = str((end-start)[0])
roll = rolling_poll.hvplot(x='time',y='Polarity Rating',rot=90, hover_cols = ['twitter_user','text'])
pol = polarity_test.hvplot(x='time',y='Polarity Rating',rot=90, hover_cols = ['twitter_user','text'],xticks=6,xlabel=f"Duration length = {recent_tweets_duration}")
polarity_test.mean()
pol

NameError: name 'polarity_test' is not defined

In [12]:
def tweet_search_free(string_query, number_tweets = 100):
    for tweets in tweepy.Cursor(api.search, q=string_query).items(number_tweets):
        screen_name = tweets.user.screen_name
        contents = ""
        #if tweets.truncated == True:
        #    contents = tweets.full_text
        #else:
        contents = tweets.text
        polarity = sentiment_reader(contents)
        
        category = category_key(contents,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), str(tweets.coordinates), contents, tweets.id, tweets.source,"x",
               "x", tweets.retweet_count, tweets.favorite_count, polarity, (polarity * int(tweets.retweet_count))]

In [13]:
#Function for generating a list of categories from the presence of keywords in text.
def category_key(text_block,dict_keywords):
    category_list = []
    output = " "
    contents = text_block.lower()
    for key in dict_keywords:
        for keyword in dict_keywords[key]:
            if keyword in contents:
                category_list.append(key)
    #return output.join(category_list)
    return category_list

In [14]:
#Function for generating sentiment values using textblob library.
def sentiment_reader(text_block):
    sentiment_text = TextBlob(text_block)
    return sentiment_text.polarity

In [15]:
#Function for fixing duplicate values in the category column.
def no_dupes(string):
    string =  str(string)
    if string[-1] != " ":
        string = string + " "
    wordlist = []
    output = ""
    word = ""
    for letter in string:
        if letter != " ":
            word = word + letter
        elif word not in wordlist:
            wordlist.append(word)
            output = output + word + " "
            word = ""
        else:
            word = ""
    return output

In [16]:
#Function for creating search query string for tweepy.Cursor(api.search). Max query length is 128 for sandbox env.
#In it's current form it supports a bunch of keywords joined by OR, grouped by (), AND from a single tweeter.
def query_creator(list_keywords, twitter_user = "!", mood = "!"):
    query = "("
    for keyword in (list_keywords):
        query += keyword
        if keyword != list_keywords[-1]:
            query += " OR "
        else:
            query += ")"
    if twitter_user != "!":
        query += " from:" + twitter_user
    return query

In [17]:
#Function for appending search results to the tweet df.
def tweet_dataframe_append(rows, target_dataframe):
    for row in rows:
        series_result = pd.Series(row, index=target_dataframe.columns)
        target_dataframe = target_dataframe.append(series_result, ignore_index=True)
        return target_dataframe

# Micheal's add in

In [18]:
elon_data = Path('../data/clean_data/clean_elon_data.csv')
elon_data_df = pd.read_csv(elon_data, index_col='date')

In [19]:
def polarity_volume_elon():
    return elon_data_df.hvplot.bar(x='date', y=['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change'], rot=90) + elon_data_df.hvplot.bar(x='date', y='Polarity Rating', rot=90)
polarity_volume_elon()

:Layout
   .Bars.I  :Bars   [date,Variable]   (value)
   .Bars.II :Bars   [date]   (Polarity Rating)

In [20]:
def popularity_volume_elon():
    return elon_data_df.hvplot.scatter(x='date', y='Popularity Rating', width=1000, height=500, yformatter='%.0f') * elon_data_df.hvplot.line(x='date', y='BTC_rolling_volume') * elon_data_df.hvplot.area(x='date', y='BTC_volume', stacked=False)
popularity_volume_elon()

:Overlay
   .Scatter.I :Scatter   [date]   (Popularity Rating)
   .Curve.I   :Curve   [date]   (BTC_rolling_volume)
   .Area.I    :Area   [date]   (BTC_volume)

In [21]:
def daily_change_elon():
    return elon_data_df.hvplot.scatter(y='Polarity Rating', x='date') * elon_data_df.hvplot(x='date', y='BTC_daily_change')
daily_change_elon()

:Overlay
   .Scatter.I :Scatter   [date]   (Polarity Rating)
   .Curve.I   :Curve   [date]   (BTC_daily_change)

In [22]:
def volume_change():
    return elon_data_df.hvplot.bar(y=['Polarity Rating', 'BTC_volume_change'], x='date', rot=45)
volume_change()

:Bars   [date,Variable]   (value)